In [54]:
# Fundamentos de Ciência de Dados
## PPGI/UFRJ 2024.2
### Profs Sergio Serra e Jorge Zavaleta
### Aluno Ubirajara S. Santos

In [55]:
import prov
import os

In [56]:
OUTPUT_DIR = './dados/saidas'
os.makedirs(OUTPUT_DIR, exist_ok=True) 

In [57]:
# Fontes de Dados
data_sources = {
     "amostras_rochas_fluidos": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-amostras-de-rochas-e-fluidos/acervo-de-amostras/consolidacao-2023.zip",
        "type": "zip"},
     "setores_sirgas": {
        "url": "https://www.gov.br/anp/pt-br/assuntos/exploracao-e-producao-de-oleo-e-gas/estudos-geologicos-e-geofisicos/arquivos-classificacao-de-modelos-exploratorios/setores-sirgas.zip",
        "type": "zip"},
     "blocos_exploratorios": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ABLOCOS_EXPLORATORIOS_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip"},
     "campos_producao": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ACAMPOS_PRODUCAO_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip"},
     "reservas_nacionais_hc": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-estatisticos/arquivos-reservas-nacionais-de-petroleo-e-gas-natural/tabela-dados-bar-2023.xlsx",
        "type": "xlsx"},
     "pocos_perfurados_2023": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/pocos-publicos-2023.csv",
        "type": "csv"},
     "tabela_levantamentos_geoquimica": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/tabela-levantamentos-geoquimicos.csv",
        "type": "csv"},
     "levantamento_sismico_2023": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/sismicos-publicos-2023.csv",
        "type": "csv"},
     "tabela_pocos_2024": {
        "url": "./dados/entradas/Tabela_pocos_2024_Novembro_24.csv",
        "type": "csv", "sep": ";" ,"encoding": "ANSI"},
     "tabela_dados_geoquimica": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/tabela-dados-geoquimicos.csv",
        "type": "csv",
        "header": 1}
}

In [58]:
import sys, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image
import plotly
import graphviz
import unicodedata
from pathlib import Path
import platform
import importlib.metadata

def gerar_prov_outputs(doc_prov):
    entity = "EDA-PROV"
    output_file = f"{entity}.png"
    try:
        dot = prov_to_dot(doc_prov)
        # Write to PNG
        dot.write_png(output_file)
        print(f"Provenance graph generated successfully: {output_file}")
        
    except Exception as e:
        print(f"Error generating provenance graph: {e}")
        # Save the DOT file for debugging
        with open("debug.dot", "w") as f:
            f.write(dot.to_string())
        print("Saved DOT file for debugging as 'debug.dot'.")
   
    # Serialização do documento
    doc_prov.serialize(entity + ".xml", format='xml') 
    doc_prov.serialize(entity + ".ttl", format='rdf', rdf_format='ttl',encoding="utf-8")
    print("Provenance serialized as XML and TTL.")
    

def adding_namespaces(document_prov):
    # Adiciona namespaces ao documento de proveniência.
    document_prov.add_namespace('void', 'http://vocab.deri.ie/void#')
    document_prov.add_namespace('ufrj', 'https://www.ufrj.br')
    document_prov.add_namespace('schema', 'http://schema.org/')    # Dados estruturados Schema.org
    document_prov.add_namespace('prov', 'http://www.w3.org/ns/prov#')     # Padrões PROV
    document_prov.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')     # Agentes FOAF
    document_prov.add_namespace('ufrj-ppgi', 'http://www.ufrj.br/ppgi/')  # UFRJ PPGI
    document_prov.add_namespace('anp', 'https://www.gov.br/anp/pt-br')    # ANP - Agência Nacional do Petróleo, Gás Natural e Biocombustíveis
    document_prov.add_namespace('anp-dados_tec','https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/acervo-de-dados-tecnicos') # ANP - Acervo de Dados Técnicos 
    document_prov.add_namespace('petrobras','https://petrobras.com.br/')  # PETROBRAS
    document_prov.add_namespace('br','http://br.org/ns/')    # Organizações Brasileiras
    return document_prov


def escape_label(text):
    """
    Escapes special characters for Graphviz.
    Encodes text to ASCII with XML character references.
    """
    return text.encode("ascii", "xmlcharrefreplace").decode()

def get_installed_packages():
    #Retorna os pacotes instalados no ambiente com suas versões.
    try:
        return {pkg.metadata['Name']: pkg.version for pkg in importlib.metadata.distributions()}
    except ImportError:
        import pkg_resources
        return {dist.project_name: dist.version for dist in pkg_resources.working_set}

def get_system_info():
    #Retorna informações do sistema.
    return {
        "OS": platform.system(),
        "OS Version": platform.version(),
        "OS Release": platform.release(),
        "Python Version": sys.version,
        "Python Executable": sys.executable,
        "Current Working Directory": str(Path.cwd()),}

def get_used_packages():
    
    #Retorna um dicionário dos pacotes usados explicitamente no projeto e suas versões.
    
    packages = ['numpy', 'pandas', 'matplotlib', 'seaborn', 'rdflib', 'prov', 'graphviz', 
                'openpyxl', 'folium', 'pydot', 'requests', 'geopandas']  # Adicione ou remova pacotes usados
    package_versions = {}
    for package in packages:
        try:
            import importlib.metadata
            version = importlib.metadata.version(package)
            package_versions[package] = version
        except ImportError:
            print(f"Pacote {package} não encontrado.")
    return package_versions

def add_system_and_package_provenance(doc_prov):
    #Adiciona informações do sistema e pacotes ao documento de proveniência
    
    # Criar atividade para rastrear informações de sistema e pacotes
    activity_id = "ufrj:track_system_and_packages"
    tracking_activity = doc_prov.activity(activity_id, datetime.datetime.now(), None, {"prov:label": escape_label("Track system and package provenance")})

    # Associar a atividade ao agente do notebook
    if "ag-eda-ipynb" in dict_agents:
        doc_prov.wasAssociatedWith(tracking_activity, dict_agents["ag-eda-ipynb"])

    # Adicionar informações do sistema como entidades
    system_info = get_system_info()
    for key, value in system_info.items():
        sanitized_key = key.replace(" ", "_")  # Substituir espaços por _
        sys_entity = doc_prov.entity(f"schema:{sanitized_key}", {"prov:value": value})
        doc_prov.wasGeneratedBy(sys_entity, tracking_activity)

    # Adicionar pacotes usados como entidades
    used_packages = get_used_packages()
    for pkg, version in used_packages.items():
        pkg_entity = doc_prov.entity(f"schema:{pkg}", {"prov:value": version})
        doc_prov.wasGeneratedBy(pkg_entity, tracking_activity)

    return doc_prov

def create_agents(document_prov):
    
    #creating agents
    dagnts={} #cria dic
    dagnts["ag-orgbr"] = document_prov.agent("br:orgBr", {"prov:type":"prov:Organization", "foaf:name":escape_label("Oraganizações Brasileiras")})
    dagnts["ag-anp"] = document_prov.agent("anp:ANP", {"prov:type":"prov:Organization", "foaf:name":escape_label("Agência Nacional do Petróleo, Gás Natural e Biocombustíveis")})
    dagnts["ag-ufrj"] = document_prov.agent("ufrj:UFRJ", {"prov:type":"prov:Organization", "foaf:name":escape_label("Universidade Federal do Rio de Janeiro")})
    dagnts["ag-ppgi"] = document_prov.agent("ufrj:PPGI", {"prov:type":"prov:Organization", "foaf:name":escape_label("Programa de Pós Graduação em Informática")})
    dagnts["ag-greco"] = document_prov.agent("ufrj:GRECO", {"prov:type":"prov:Organization", "foaf:name":escape_label("Grupo de Engenharia do Conhecimento")})
    dagnts["ag-author-ubirajara"] = document_prov.agent("ufrj:Ubirajara", {"prov:type":"prov:Person", "foaf:name":escape_label("Ubirajara Simões Santos"), "foaf:mbox":"ubirajas@hotmail.com"})
    dagnts["ag-author-sergio"] = document_prov.agent("ufrj:Sergio", {"prov:type":"prov:Person", "foaf:name":escape_label("Sergio Serra"), "foaf:mbox":"serra@ppgi.ufrj.br"})
    dagnts["ag-author-jorge"] = document_prov.agent("ufrj:Jorge", {"prov:type":"prov:Person", "foaf:name":escape_label("Jorge Zavaleta"), "foaf:mbox":"zavaleta@pet-si.ufrrj.br"})
    dagnts["ag-petrobras"] = document_prov.agent("petrobras:Petrobras", {"prov:type":"prov:Organization", "foaf:name":escape_label("Petróleo Brasiliero S.A")})
    dagnts["ag-eda-ipynb"] = document_prov.agent("ufrj:eda.ipynb", {"prov:type":"prov:SoftwareAgent", "foaf:name":escape_label("eda.ipynb"), "prov:label":escape_label("Notebook Python utilizado no trabalho")})
    return dagnts

def associate_ufrj_agents(agents_dictionary):
    agents_dictionary["ag-anp"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-petrobras"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-ufrj"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-ppgi"].actedOnBehalfOf(agents_dictionary["ag-ufrj"])
    agents_dictionary["ag-greco"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-author-ubirajara"].actedOnBehalfOf(agents_dictionary["ag-greco"])
    agents_dictionary["ag-author-ubirajara"].actedOnBehalfOf(agents_dictionary["ag-petrobras"])
    agents_dictionary["ag-author-sergio"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-author-jorge"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-eda-ipynb"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    return agents_dictionary

 

def create_initial_activities(document_prov):
    #creating activities
    #dataDownloadDatasets = datetime.datetime.strptime('29/11/24', '%d/%m/%y')
    
    dativs={}
    dativs["act-create-ds"] = document_prov.activity("anp:create-dataset", None, None, {"prov:label":escape_label( "Criação de datasets pela ANP")})
    #dativs["act-extract-ds"] = document_prov.activity("ufrj:extract-dataset")
    dativs["act-create-ds-eda"] = document_prov.activity("ufrj:create-ds-eda", None, None, {"prov:label":escape_label( "Criação de datasets para EDA")})
    #dativs["act-load-ds-eda"] = document_prov.activity("ufrj:load-ds-eda")
    dativs["act-save-ipynb"] = document_prov.activity("ufrj:save-ipynb", None, None, {"prov:label":escape_label("Salvar notebook EDA")})
    return dativs

def cria_entidades_iniciais(document_prov):
    #creating entidades
    dents={}
    
    # Entidade para amostras de rochas e fluidos
    dents["ent-amostras-rochas-fluidos"] = document_prov.entity('anp:amostras_rochas_fluidos', {'prov:label':escape_label('Dataset com amostras de rochas e fluidos'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Consolidado 2023 de amostras disponíveis.'), 'prov:format': 'zip' })
    # Entidade para setores SIRGAS
    dents["ent-setores-sirgas"] = document_prov.entity('anp:setores_sirgas', {'prov:label':escape_label('Setores SIRGAS'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Modelos exploratórios em formato SIRGAS.'), 'prov:format': 'zip'})
    # Entidade para blocos exploratórios
    dents["ent-blocos-exploratorios"] = document_prov.entity('anp:blocos_exploratorios', {'prov:label':escape_label( 'Blocos exploratórios'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Blocos exploratórios com dados geoespaciais.'), 'prov:format': 'zip'})
    # Entidade para campos de produção
    dents["ent-campos-producao"] = document_prov.entity('anp:campos_producao', {'prov:label':escape_label( 'Campos de Produção'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados dos campos de produção em formato SIRGAS.'), 'prov:format': 'zip'})
    # Entidade para reservas nacionais de hidrocarbonetos
    dents["ent-reservas-nacionais-hc"] = document_prov.entity('anp:reservas_nacionais_hc',{'prov:label':escape_label( 'Reservas Nacionais de Hidrocarbonetos'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Tabela com dados sobre reservas nacionais.'), 'prov:format': 'xlsx'})
    # Entidade para poços perfurados (2023)
    dents["ent-pocos-perfurados-2023"] = document_prov.entity('anp:pocos_perfurados_2023',{'prov:label':escape_label( 'Poços perfurados - 2023'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('CSV com os poços perfurados no ano de 2023.'), 'prov:format': 'csv'})
    # Entidade para tabela de levantamentos geoquímicos
    dents["ent-tabela-levantamentos-geoquimica"] = document_prov.entity('anp:tabela_levantamentos_geoquimica',{'prov:label':escape_label( 'Tabela de levantamentos geoquímicos 20/04/2022'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados sobre levantamentos geoquímicos.'), 'prov:format': 'csv'})
     # Entidade para tabela de dados geoquímicos
    dents["ent-tabela-dados-geoquimica"] = document_prov.entity('anp:tabela_dados_geoquimica',{'prov:label':escape_label( 'Tabela_dados_geoquimica 06/08/2021'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados geoquímicos.'), 'prov:format': 'csv'})
     # Entidade para levantamento sísmico (2023)
    dents["ent-levantamento-sismico-2023"] = document_prov.entity('anp:levantamento_sismico_2023', {'prov:label':escape_label( 'Levantamento Sísmico - 2023'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('CSV com dados de levantamentos sísmicos públicos.'), 'prov:format': 'csv'})
    # Entidade para tabela de poços (2024)
    dents["ent-tabela-pocos-2024"] = document_prov.entity('anp:tabela_pocos_2024', {'prov:label':escape_label( 'Tabela de Poços - 2024'.encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Tabela CSV com dados atualizados de poços para 2024.'), 'prov:format': 'csv'})
     # Entidade para ANP dados técnicos
    dents["ent-anp-dados_tec-ds"] = document_prov.entity('anp-dados_tec:dataset', {'prov:label':escape_label( 'ANP Dataset de Dados Técnicos'.encode("ascii", "xmlcharrefreplace").decode()),'prov:type': 'void:Dataset','prov:description':escape_label('Dataset com dados técnicos disponíveis publicamente.'),'prov:format': 'csv'})
    
    # Entidade script python
    dents["ent-eda-ipynb"] = document_prov.entity('ufrj:eda-ipyn', {'prov:label':escape_label( "Notebook Python utilizado no trabalho".encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'foaf:Document'})
    # Entidade Git
    dents["ent-git-eda"] = document_prov.entity('anp:github-eda', {'prov:label':escape_label( 'Repositorio Eba da ANP'.encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'prov:Collection'})
    return dents
  

def initial_association_agents_activities_entities(document_prov, dictionary_agents, dictionary_activities, dictionary_entities):
    
    #Associate activity of generate dataset with ANP agent
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds"], dictionary_agents["ag-anp"])
    
    #Associating datasets with activities of generate eba datasets
    document_prov.wasGeneratedBy(dictionary_entities["ent-amostras-rochas-fluidos"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-setores-sirgas"], dictionary_activities["act-create-ds"])    
    document_prov.wasGeneratedBy(dictionary_entities["ent-blocos-exploratorios"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-campos-producao"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-reservas-nacionais-hc"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-pocos-perfurados-2023"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-levantamentos-geoquimica"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-dados-geoquimica"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-levantamento-sismico-2023"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-pocos-2024"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-anp-dados_tec-ds"], dictionary_activities["act-create-ds"])
    
    
    #Associating ZIPs, XLSX, CSV com entities do dataset genérico
    #document_prov.wasDerivedFrom(dictionary_entities["ent-dredfp2021-zip"], dictionary_entities["ent-dredfp"])  
       
    #associate activity of eda, com autor
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds-eda"], dictionary_agents["ag-author-ubirajara"])   

    #associate notebook agent with eba dataset
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds-eda"], dictionary_agents["ag-eda-ipynb"])    
             
    #associate eda github repository with store datasets activity
    document_prov.wasGeneratedBy(dictionary_entities["ent-git-eda"], dictionary_activities["act-save-ipynb"])

def associate_save_activity(doc_prov, dict_agents, dict_entities):
    
    #Associa a atividade de salvar notebook ao agente e à entidade relevante.
   
    activity_id = "ufrj:save-ipynb"
    save_activity = doc_prov.activity(activity_id, datetime.datetime.now(), None, {"prov:label": escape_label("Salvar notebook EDA")})
    # Associar ao agente eda.ipynb
    if "ag-eda-ipynb" in dict_agents:
        doc_prov.wasAssociatedWith(save_activity, dict_agents["ag-eda-ipynb"])

    # Associar à entidade eda-ipynb
    if "ent-eda-ipynb" in dict_entities:
        doc_prov.wasGeneratedBy(dict_entities["ent-eda-ipynb"], save_activity)

    return doc_prov
    
    
def initProvenance():
    #Inicializa o documento de proveniência com namespaces, agentes, atividades e entidades.
    
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Criando um documento vazio de proveniência
    doc_prov = ProvDocument()

    # Criar namespaces no documento de proveniência
    doc_prov = adding_namespaces(doc_prov)

    # Criar agentes
    dict_agents = create_agents(doc_prov)

    # Criar atividades iniciais
    dict_activities = create_initial_activities(doc_prov)

    # Criar entidades iniciais
    dict_entities = cria_entidades_iniciais(doc_prov)

    # Criar hierarquia de agentes
    dict_agents = associate_ufrj_agents(dict_agents)

    # Associar agentes, atividades e entidades
    initial_association_agents_activities_entities(doc_prov, dict_agents, dict_activities, dict_entities)

    # Adicionar proveniência do sistema e pacotes
    doc_prov = add_system_and_package_provenance(doc_prov)

    # Associar atividade ufrj:save-ipynb
    doc_prov = associate_save_activity(doc_prov, dict_agents, dict_entities)

    return doc_prov, dict_agents, dict_activities, dict_entities


    


In [59]:
import pandas as pd
import matplotlib
import openpyxl
from io import BytesIO
import requests
import zipfile
import datetime


In [60]:
def analyze_zip_content(url, temp_dir="./dados/temp"):
    """
    Analisa o conteúdo de um arquivo ZIP e categoriza os tipos de arquivos encontrados.

    Args:
        url (str): URL para o arquivo ZIP.
        temp_dir (str): Diretório temporário para extração dos arquivos.

    Returns:
        dict: Dicionário com categorias de arquivos (csv, xlsx, shp, others).
    """
    os.makedirs(temp_dir, exist_ok=True)
    file_types = {"csv": [], "xlsx": [], "xls": [], "shp": [], "others": []}

    try:
        # Baixar o arquivo ZIP
        response = requests.get(url)
        if response.status_code != 200:
            raise ValueError(f"Erro ao baixar o arquivo: {response.status_code}")

        # Abrir o ZIP em memória e extrair
        with zipfile.ZipFile(BytesIO(response.content)) as zf:
            zf.extractall(temp_dir)
            extracted_files = zf.namelist()

        # Categorizar os arquivos extraídos
        for file in extracted_files:
            file_path = os.path.join(temp_dir, file)
            if file.endswith(".csv"):
                file_types["csv"].append({"name": os.path.basename(file), "path": file_path})
            elif file.endswith(".xlsx"):
                file_types["xlsx"].append({"name": os.path.basename(file), "path": file_path})
            elif file.endswith(".xls"):
                file_types["xls"].append({"name": os.path.basename(file), "path": file_path})
            elif file.endswith(".shp"):
                file_types["shp"].append({"name": os.path.basename(file), "path": file_path})
            else:
                file_types["others"].append({"name": os.path.basename(file), "path": file_path})

        print(f"Conteúdo do ZIP analisado: {file_types}")
        return file_types

    except zipfile.BadZipFile:
        print(f"O arquivo fornecido não é um ZIP válido: {url}")
        return None
    except Exception as e:
        print(f"Erro ao processar ZIP: {e}")
        return None

In [61]:
def diagnose_csv(file_path, encodings=["utf-8", "ISO-8859-1", "utf-8-sig"]):
    """
    Diagnostica problemas em arquivos CSV: delimitador e encoding.
    """
    print(f"Diagnóstico do arquivo: {file_path}")
    for encoding in encodings:
        try:
            print(f"Tentando com encoding: {encoding}")
            with open(file_path, "r", encoding=encoding) as f:
                sample = f.read(2048)  # Lê os primeiros 2048 caracteres
            print(f"Primeiros caracteres ({encoding}):")
            print(sample[:500])  # Mostra apenas os primeiros 500 caracteres
            print("\n--- Fim da Amostra ---\n")
        except Exception as e:
            print(f"Erro ao ler o arquivo com encoding {encoding}: {e}")

In [62]:
def diagnose_csv(file_path, encodings=["utf-8", "ISO-8859-1", "utf-8-sig"], max_lines=5):
    """
    Diagnostica problemas em arquivos CSV: delimitador e encoding.

    Args:
        file_path (str): Caminho do arquivo CSV a ser diagnosticado.
        encodings (list): Lista de encodings para tentar.
        max_lines (int): Número máximo de linhas a serem lidas para análise.

    Returns:
        dict: Informações diagnosticadas incluindo encoding funcional e conteúdo inicial.
    """
    print(f"=== Diagnóstico do arquivo: {file_path} ===")
    diagnostics = {"file_path": file_path, "encoding": None, "sample": None, "error": None}

    for encoding in encodings:
        try:
            print(f"Tentando com encoding: {encoding}")
            with open(file_path, "r", encoding=encoding) as f:
                sample = [f.readline().strip() for _ in range(max_lines)]

            diagnostics["encoding"] = encoding
            diagnostics["sample"] = sample
            print(f"Arquivo lido com sucesso usando encoding '{encoding}'.")
            print(f"Amostra das primeiras {max_lines} linhas:")
            print("\n".join(sample))
            print("\n--- Fim da Amostra ---\n")
            return diagnostics  # Retorna na primeira tentativa bem-sucedida

        except Exception as e:
            print(f"Erro ao ler o arquivo com encoding {encoding}: {e}")
            diagnostics["error"] = str(e)

    # Retorna informações de erro se nenhum encoding funcionar
    print("Falha ao diagnosticar o arquivo com os encodings fornecidos.")
    return diagnostics

In [63]:
def process_xlsx_files(xlsx_files):
    """
    Carrega arquivos XLSX em DataFrames e retorna um dicionário de DataFrames.

    Args:
        xlsx_files (list): Lista de dicionários com informações dos arquivos XLSX. Cada dicionário contém:
            - 'name': Nome do arquivo.
            - 'path': Caminho para o arquivo.

    Returns:
        dict: Um dicionário onde as chaves são os nomes dos arquivos e os valores são os DataFrames.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities  # Para rastrear proveniência

    dataframes = {}
    for xlsx_info in xlsx_files:
        try:
            # Início da atividade de carga
            exec_start = datetime.datetime.now()
            
            # Carregar o arquivo XLSX
            df = pd.read_excel(xlsx_info["path"])
            dataframes[xlsx_info["name"]] = df

            # Rastrear proveniência
            exec_end = datetime.datetime.now()
            activity_key = f"act-load-xlsx-{xlsx_info['name']}"
            dict_activities[activity_key] = doc_prov.activity(
                f"ufrj:load_xlsx_{xlsx_info['name']}",
                exec_start,
                exec_end,
                {"prov:label": escape_label(f"Carregamento do arquivo XLSX: {xlsx_info['name']}")}
            )
            doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

            entity_key = f"ent-xlsx-{xlsx_info['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:{xlsx_info['name']}",
                {"prov:label": escape_label(f"Arquivo XLSX carregado: {xlsx_info['name']}"),
                 "prov:type": "void:Dataset",
                 "prov:generatedAtTime": exec_end.isoformat()}
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

            print(f"Arquivo XLSX carregado: {xlsx_info['name']} com shape {df.shape}")
        
        except Exception as e:
            print(f"Erro ao carregar XLSX {xlsx_info['name']}: {e}")

    return dataframes

In [64]:
import geopandas as gpd

def process_shp_files(shp_files):
    """
    Carrega arquivos Shapefiles em GeoDataFrames e retorna um dicionário de GeoDataFrames.

    Args:
        shp_files (list): Lista de dicionários com informações dos arquivos Shapefile.
            Cada dicionário contém:
                - 'name': Nome do arquivo.
                - 'path': Caminho completo para o arquivo.

    Returns:
        dict: Um dicionário onde as chaves são os nomes dos arquivos e os valores são os GeoDataFrames.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities  # Para rastrear proveniência

    geodataframes = {}
    for shp_info in shp_files:
        try:
            # Início da atividade de carga
            exec_start = datetime.datetime.now()
            
            # Carregar o arquivo Shapefile em um GeoDataFrame
            gdf = gpd.read_file(shp_info["path"])
            geodataframes[shp_info["name"]] = gdf

            # Rastrear proveniência
            exec_end = datetime.datetime.now()
            activity_key = f"act-load-shp-{shp_info['name']}"
            dict_activities[activity_key] = doc_prov.activity(
                f"ufrj:load_shp_{shp_info['name']}",
                exec_start,
                exec_end,
                {"prov:label": escape_label(f"Carregamento do Shapefile: {shp_info['name']}")}
            )
            doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

            entity_key = f"ent-shp-{shp_info['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:{shp_info['name']}",
                {"prov:label": escape_label(f"Shapefile carregado: {shp_info['name']}"),
                 "prov:type": "void:Dataset",
                 "prov:generatedAtTime": exec_end.isoformat()}
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

            print(f"Arquivo Shapefile carregado: {shp_info['name']} com shape {gdf.shape}")
        
        except Exception as e:
            print(f"Erro ao carregar Shapefile {shp_info['name']}: {e}")

    return geodataframes

In [65]:
def report_unknown_files(unknown_files):
    """
    Exibe uma mensagem com arquivos de formatos não reconhecidos e registra a proveniência.

    Args:
        unknown_files (list): Lista de dicionários com informações sobre arquivos não reconhecidos.
            Cada dicionário contém:
                - 'name': Nome do arquivo.
                - 'path': Caminho completo para o arquivo.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities  # Proveniência

    if unknown_files:
        print(f"Arquivos não reconhecidos encontrados: {[file['name'] for file in unknown_files]}")

        # Registrar proveniência da análise
        exec_start = datetime.datetime.now()
        activity_key = "act-analyze-unknown-files"
        dict_activities[activity_key] = doc_prov.activity(
            "ufrj:analyze_unknown_files",
            exec_start,
            None,
            {"prov:label": escape_label("Análise de arquivos desconhecidos")}
        )
        doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

        # Criar entidades para cada arquivo desconhecido
        for file_info in unknown_files:
            entity_key = f"ent-unknown-{file_info['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:unknown_{file_info['name']}",
                {"prov:label": escape_label(f"Arquivo desconhecido: {file_info['name']}"),
                 "prov:type": "void:Dataset",
                 "prov:location": file_info["path"]}
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

    else:
        print("Nenhum arquivo não reconhecido foi encontrado.")

In [66]:
def process_zip_source(source_name, data_sources, temp_dir="./dados/temp"):
    # Proveniência
    global doc_prov, dict_agents, dict_activities, dict_entities

    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return {"dataframes": {}, "geodataframes": {}}

    exec_start = datetime.datetime.now()

    # Analisar conteúdo do ZIP
    file_types = analyze_zip_content(source["url"], temp_dir=temp_dir)
    if not file_types:
        print(f"Erro ao analisar o conteúdo do ZIP '{source_name}'.")
        return {"dataframes": {}, "geodataframes": {}}

    # Criar atividade de processamento de ZIP
    activity_key = f"act-process-zip-{source_name}"
    dict_activities[activity_key] = doc_prov.activity(
        f"ufrj:process_zip_{source_name}",
        exec_start,
        None,
        {"prov:label": escape_label(f"Processamento do ZIP: {source_name}")}
    )
    doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

    # Associar a entidade do ZIP original à atividade
    zip_entity_key = f"ent-{source_name}"
    if zip_entity_key in dict_entities:
        doc_prov.used(dict_activities[activity_key], dict_entities[zip_entity_key])

    # Inicializar coleções
    dataframes = {}
    geodataframes = {}

    # Processar CSVs
    for csv_file in file_types["csv"]:
        try:
            df = pd.read_csv(csv_file["path"], sep=",", encoding="utf-8")
            dataframes[csv_file["name"]] = df

            # Criar entidade para cada CSV
            entity_key = f"ent-csv-{csv_file['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:csv_{csv_file['name']}",
                {
                    "prov:label": escape_label(f"CSV carregado: {csv_file['name']}"),
                    "prov:type": "void:Dataset",
                    "prov:generatedAtTime": datetime.datetime.now().isoformat(),
                }
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
            print(f"CSV carregado: {csv_file['name']} com shape {df.shape}")

        except Exception as e:
            print(f"Erro ao carregar CSV {csv_file['name']}: {e}")

    # Processar XLSXs
    for xlsx_file in file_types["xlsx"]:
        try:
            df = pd.read_excel(xlsx_file["path"])
            dataframes[xlsx_file["name"]] = df

            # Criar entidade para cada XLSX
            entity_key = f"ent-xlsx-{xlsx_file['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:xlsx_{xlsx_file['name']}",
                {
                    "prov:label": escape_label(f"XLSX carregado: {xlsx_file['name']}"),
                    "prov:type": "void:Dataset",
                    "prov:generatedAtTime": datetime.datetime.now().isoformat(),
                }
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
            print(f"XLSX carregado: {xlsx_file['name']} com shape {df.shape}")

        except Exception as e:
            print(f"Erro ao carregar XLSX {xlsx_file['name']}: {e}")

    # Processar Shapefiles
    for shp_file in file_types["shp"]:
        try:
            gdf = gpd.read_file(shp_file["path"])
            geodataframes[shp_file["name"]] = gdf

            # Criar entidade para cada Shapefile
            entity_key = f"ent-shp-{shp_file['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:shp_{shp_file['name']}",
                {
                    "prov:label": escape_label(f"Shapefile carregado: {shp_file['name']}"),
                    "prov:type": "void:Dataset",
                    "prov:generatedAtTime": datetime.datetime.now().isoformat(),
                }
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
            print(f"Shapefile carregado: {shp_file['name']} com shape {gdf.shape}")

        except Exception as e:
            print(f"Erro ao carregar Shapefile {shp_file['name']}: {e}")

    # Relatar arquivos não reconhecidos
    report_unknown_files(file_types["others"])

    # Finalizar atividade
    exec_end = datetime.datetime.now()
    dict_activities[activity_key].add_attributes({"prov:endTime": exec_end.isoformat()})

    return {"dataframes": dataframes, "geodataframes": geodataframes}

In [67]:
import geopandas as gpd
import os
import zipfile
from io import BytesIO
import datetime

def extract_and_load_shapefile_from_zip(source_name, data_sources, temp_dir="./dados/temp"):
    """
    Extrai e carrega Shapefiles de um ZIP com rastreamento de proveniência.
    Args:
        source_name (str): Nome da fonte de dados no `data_sources`.
        data_sources (dict): Dicionário contendo informações das fontes de dados.
        temp_dir (str): Diretório temporário para extração dos arquivos.
    Returns:
        dict: Dicionário contendo informações sobre os Shapefiles e seus GeoDataFrames.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities  # Proveniência

    os.makedirs(temp_dir, exist_ok=True)
    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return {}

    exec_start = datetime.datetime.now()
    shapefile_info = {}

    try:
        # Baixar o arquivo ZIP
        response = requests.get(source["url"])
        if response.status_code != 200:
            raise ValueError(f"Erro ao baixar o arquivo: {response.status_code}")

        # Abrir o ZIP em memória e extrair
        with zipfile.ZipFile(BytesIO(response.content)) as zf:
            zf.extractall(temp_dir)
            extracted_files = zf.namelist()

        # Procurar arquivos .shp
        shp_files = [file for file in extracted_files if file.endswith(".shp")]

        if not shp_files:
            raise ValueError("Nenhum arquivo Shapefile (.shp) encontrado no ZIP.")

        # Criar atividade de extração no documento de proveniência
        activity_key = f"act-extract-shp-{source_name}"
        dict_activities[activity_key] = doc_prov.activity(
            f"ufrj:extract_shp_{source_name}",
            exec_start,
            None,
            {"prov:label": escape_label(f"Extração de Shapefile do ZIP: {source_name}")}
        )
        doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

        # Associar atividade ao ZIP original
        zip_entity_key = f"ent-{source_name}"
        if zip_entity_key in dict_entities:
            doc_prov.used(dict_activities[activity_key], dict_entities[zip_entity_key])

        # Processar todos os Shapefiles encontrados
        for shp_file in shp_files:
            shapefile_path = os.path.join(temp_dir, shp_file)
            gdf = gpd.read_file(shapefile_path)

            # Criar entidade para cada Shapefile
            entity_key = f"ent-shp-{source_name}-{os.path.basename(shp_file)}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:{source_name}_{os.path.basename(shp_file)}",
                {
                    "prov:label": escape_label(f"Shapefile extraído: {os.path.basename(shp_file)}"),
                    "prov:type": "void:Dataset",
                    "prov:generatedAtTime": datetime.datetime.now().isoformat(),
                }
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

            # Armazenar informações do Shapefile
            shapefile_info[os.path.basename(shp_file)] = {
                "path": shapefile_path,
                "crs": gdf.crs,
                "columns": gdf.columns.tolist(),
                "geometry_type": gdf.geom_type.unique(),
                "gdf": gdf
            }
            print(f"Shapefile carregado: {shapefile_path} com shape {gdf.shape}")

        exec_end = datetime.datetime.now()
        dict_activities[activity_key].add_attributes({"prov:endTime": exec_end.isoformat()})

        return shapefile_info

    except Exception as e:
        print(f"Erro ao processar Shapefile no ZIP '{source_name}': {e}")
        return {}

In [68]:
def update_data_sources_with_zip(data_sources, source_name, temp_dir="./dados/temp"):
    """
    Atualiza `data_sources` com os arquivos extraídos de um ZIP, criando uma relação hierárquica.
    """
    # Obter fonte original
    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return data_sources

    # Analisar o conteúdo do ZIP
    file_types = analyze_zip_content(source["url"], temp_dir=temp_dir)
    if not file_types:
        print(f"Erro ao processar ZIP '{source_name}'.")
        return data_sources

    # Criar relações de proveniência
    for file_type, files in file_types.items():
        if not files:
            continue

        for i, file_info in enumerate(files):
            child_name = f"{source_name}_{file_type}_{i+1}"
            data_sources[child_name] = {
                "url": file_info["path"],  # Caminho local do arquivo extraído
                "type": file_type,        # Tipo do arquivo (csv, xlsx, shp, etc.)
                "parent": source_name,    # Referência ao ZIP pai
                "sep": source.get("sep", ";") if file_type == "csv" else None,
                "encoding": source.get("encoding", "utf-8") if file_type == "csv" else None,
                "header": source.get("header", 0) if file_type == "csv" else None,
                "date_columns": source.get("date_columns", []) if file_type == "csv" else None,
            }
            print(f"Adicionado {child_name} como filho de {source_name}.")
    
    print(f"Data sources atualizados com arquivos de '{source_name}'.")
    return data_sources

In [69]:
def update_data_sources_with_zip(data_sources, source_name, temp_dir="./temp"):
    """
    Atualiza `data_sources` com os arquivos extraídos de um ZIP, criando uma relação hierárquica.
    E registra a proveniência entre o ZIP (pai) e os arquivos extraídos (filhos).
    """
    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return data_sources

    # Analisar o conteúdo do ZIP
    file_types = analyze_zip_content(source["url"], temp_dir=temp_dir)
    if not file_types:
        print(f"Erro ao processar ZIP '{source_name}'.")
        return data_sources

    # Lista de filhos (arquivos extraídos)
    children_sources = []

    for file_type, files in file_types.items():
        if not files:
            continue

        for i, file_info in enumerate(files):
            child_name = f"{source_name}_{file_type}_{i+1}"
            data_sources[child_name] = {
                "url": file_info["path"],  # Caminho local do arquivo extraído
                "type": file_type,        # Tipo do arquivo (csv, xlsx, shp, etc.)
                "parent": source_name,    # Referência ao ZIP pai
                "sep": source.get("sep", ";") if file_type == "csv" else None,
                "encoding": source.get("encoding", "utf-8") if file_type == "csv" else None,
                "header": source.get("header", 0) if file_type == "csv" else None,
                "date_columns": source.get("date_columns", []) if file_type == "csv" else None,
            }
            children_sources.append(child_name)
            print(f"Adicionado {child_name} como filho de {source_name}.")

    # Registrar a proveniência para o ZIP e seus filhos
    register_provenance_for_zip_and_children(source_name, children_sources)
    
    print(f"Data sources atualizados com arquivos de '{source_name}'.")
    return data_sources

In [70]:
def register_provenance_for_zip_and_children(parent_source, children_sources, activity_prefix="process-zip"):
    """
    Registra a proveniência entre o ZIP pai e os arquivos extraídos (filhos).
    A atividade é associada ao agente 'ag-eda-ipynb' para todos os filhos.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Criar atividade para o processamento do ZIP
    exec_start = datetime.datetime.now()
    activity_key = f"{activity_prefix}-{parent_source}"
    dict_activities[activity_key] = doc_prov.activity(
        f"ufrj:{activity_prefix}_{parent_source}", exec_start, None,
        {"prov:label": escape_label(f"Processamento do ZIP {parent_source}")}
    )
    doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

    # Registrar cada filho como derivado do pai
    for child_source in children_sources:
        entity_key = f"ent-{child_source}"
        dict_entities[entity_key] = doc_prov.entity(f"ufrj:{child_source}", {
            "prov:label": escape_label(f"Arquivo derivado de {parent_source}"),
            "prov:type": "void:Dataset"
        })
        
        # Relacionar pai e filho
        doc_prov.wasDerivedFrom(
            dict_entities[entity_key], dict_entities.get(f"ent-{parent_source}")
        )
        doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

        # Adiciona a atividade de proveniência do arquivo como "gerado por" o agente IPYNB
        doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

    print(f"Proveniência registrada para arquivos derivados de {parent_source}.")


In [90]:
def load_data_from_source_csv(source_name, data_sources):
    """
    Carrega dados com base no nome da fonte e na configuração em data_sources.
    Returns:
        pd.DataFrame: DataFrame com os dados carregados, ou None se houver erro.
    """
    global doc_prov, dict_agents,  dict_activities, dict_entities  # Declare global variables
    #save execution start time
    execStartTime = datetime.datetime.now()

    source = data_sources.get(source_name)
   
    if not source: #or source.get("type") != "csv":
        print(f"Fonte '{source_name}' não encontrada ou não é do tipo CSV.")
        return None

    file_type = source.get("type")
    url = source.get("url")
    sep = source.get("sep", ";")  # Valor padrão para CSV
    encoding = source.get("encoding", "utf-8")  # Valor padrão para codificação
    date_columns = source.get("date_columns", [])  

    try:
        if file_type == "csv":
             # Caso específico para tabela_pocos_2024
            if source_name == "tabela_pocos_2024":
                df = pd.read_csv(url, encoding="ANSI", sep=sep)
            # Caso específico para tabela_dados_geoquimica
            elif source_name == "tabela_dados_geoquimica":
                df = pd.read_csv(url, sep=sep, encoding=encoding, header=1)  # Cabeçalho na segunda linha
            else:
                #header_line = detect_csv_header(url, delimiter=sep)
                df = pd.read_csv(url, sep=sep, encoding=encoding, parse_dates=date_columns)
        elif file_type == "xlsx":
            df = pd.read_excel(url)
        else:
            print(f"Tipo de arquivo '{file_type}' não suportado.")
            return None
        print(f"Dados carregados com sucesso para '{source_name}'.")
    
        # End execution time for provenance tracking
        execEndTime = datetime.datetime.now()
    
        # Criar atividade com horário de término da execução
        activity_key = f"act-carga-{source_name}"
        dict_activities[activity_key] = doc_prov.activity(f"ufrj:carga_{source_name}", execStartTime, execEndTime)
    
        # Associar a atividade ao agente
        doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])
        
        # Associar a atividade com os dados carregados
        entity_key = f"ent-{source_name}"
        dict_entities[entity_key] = doc_prov.entity(f"ufrj:{source_name}", {
            "prov:label": escape_label(f"Dataset carregado: {source_name}"),"prov:type": "void:Dataset", "prov:generatedAtTime": execEndTime.isoformat(),})
        doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
    
        # Associar a atividade ufrj:carga à entidade correspondente criada pela ANP
        anp_entity_key = f"ent-{source_name.replace('_', '-')}"  # Convert to ANP format (e.g., `tabela_pocos_2024` -> `ent-tabela-pocos-2024`)
        if anp_entity_key in dict_entities:
            # Establish the prov:used relationship
            doc_prov.used(dict_activities[activity_key], dict_entities[anp_entity_key])
        else:
            print(f"Warning: ANP entity '{anp_entity_key}' not found for activity '{activity_key}'.")
        return df
        
    except Exception as e:
        print(f"Erro ao carregar dados de '{source_name}': {e}")
        return None

In [91]:
def process_zip_source_shp(source_name, data_sources, temp_dir="./dados/temp"):
    # Processa um ZIP contendo Shapefiles com rastreamento de proveniência.
    global doc_prov, dict_agents, dict_activities, dict_entities  # Proveniência

    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return {"geodataframes": {}}

    exec_start = datetime.datetime.now()

    # Analisar conteúdo do ZIP
    file_types = analyze_zip_content(source["url"], temp_dir=temp_dir)
    if not file_types:
        print(f"Erro ao analisar o conteúdo do ZIP '{source_name}'.")
        return {"geodataframes": {}}

    # Criar atividade de processamento de ZIP
    activity_key = f"act-process-zip-{source_name}"
    dict_activities[activity_key] = doc_prov.activity(
        f"ufrj:carga_{source_name}",
        exec_start,
        None,
        {"prov:label": escape_label(f"Processamento do ZIP: {source_name}")}
    )
    doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

    # Associar a entidade do ZIP original à atividade
    zip_entity_key = f"ent-{source_name}"
    if zip_entity_key in dict_entities:
        doc_prov.used(dict_activities[activity_key], dict_entities[zip_entity_key])

    # Criar a entidade derivada `ufrj:<source_name>`
    derived_entity_key = f"ent-{source_name}-ufrj"
    dict_entities[derived_entity_key] = doc_prov.entity(
        f"ufrj:{source_name}",
        {
            "prov:label": escape_label(f"Dataset processado: {source_name}"),
            "prov:type": "void:Dataset",
            "prov:generatedAtTime": datetime.datetime.now().isoformat(),
        }
    )
    doc_prov.wasGeneratedBy(dict_entities[derived_entity_key], dict_activities[activity_key])

    # Processar Shapefiles
    geodataframes = {}
    for shp_file in file_types["shp"]:
        try:
            gdf = gpd.read_file(shp_file["path"])
            geodataframes[shp_file["name"]] = gdf

            # Criar entidade para cada Shapefile
            entity_key = f"ent-shp-{shp_file['name']}"
            dict_entities[entity_key] = doc_prov.entity(
                f"ufrj:shp_{shp_file['name']}",
                {
                    "prov:label": escape_label(f"Shapefile carregado: {shp_file['name']}"),
                    "prov:type": "void:Dataset",
                    "prov:generatedAtTime": datetime.datetime.now().isoformat(),
                }
            )
            doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
            print(f"Shapefile carregado: {shp_file['name']} com shape {gdf.shape}")

        except Exception as e:
            print(f"Erro ao carregar Shapefile {shp_file['name']}: {e}")

    # Relatar arquivos não reconhecidos
    report_unknown_files(file_types["others"])

    exec_end = datetime.datetime.now()
    dict_activities[activity_key].add_attributes({"prov:endTime": exec_end.isoformat()})

    return {"geodataframes": geodataframes}


In [92]:
def save_dataframes(globals_dict, output_dir="./dados/saidas"):
    """
    Salva todos os DataFrames e GeoDataFrames do escopo global no diretório de saída.
    """
    os.makedirs(output_dir, exist_ok=True)
    saved_objects = []

    for obj_name, obj in globals_dict.items():
        try:
            if isinstance(obj, pd.DataFrame):
                file_path = os.path.join(output_dir, f"{obj_name}.csv")
                obj.to_csv(file_path, index=False)
                print(f"DataFrame '{obj_name}' salvo em '{file_path}'.")
                saved_objects.append(obj_name)

            elif isinstance(obj, gpd.GeoDataFrame):
                file_path = os.path.join(output_dir, f"{obj_name}.geojson")
                obj.to_file(file_path, driver="GeoJSON")
                print(f"GeoDataFrame '{obj_name}' salvo em '{file_path}'.")
                saved_objects.append(obj_name)
        except Exception as e:
            print(f"Erro ao salvar '{obj_name}': {e}")

    return saved_objects

In [93]:
def create_eda_dataset():
    global df_sismica_2023_orig, df_pocos_orig, df_lev_geoq_2022_orig, df_geoq_2021_orig, df_reservas_orig, df_poco_2023_orig
    global gdf_setores_sirgas, gdf_blocos_exploratorios, gdf_campos_producao

    # Carregar datasets CSV/XLSX
    df_sismica_2023_orig = load_data_from_source_csv("levantamento_sismico_2023", data_sources)
    df_pocos_orig = load_data_from_source_csv("tabela_pocos_2024", data_sources)
    df_lev_geoq_2022_orig = load_data_from_source_csv("tabela_levantamentos_geoquimica", data_sources)
    df_geoq_2021_orig = load_data_from_source_csv("tabela_dados_geoquimica", data_sources)
    df_reservas_orig = load_data_from_source_csv("reservas_nacionais_hc", data_sources)
    df_poco_2023_orig = load_data_from_source_csv("pocos_perfurados_2023", data_sources)

    # Processar shapefiles do ZIP
    gdf_setores_sirgas = process_zip_source_shp("setores_sirgas", data_sources).get("geodataframes", {}).get("SETORES_TODOS_SIRGAS.shp")
    gdf_blocos_exploratorios = process_zip_source_shp("blocos_exploratorios", data_sources).get("geodataframes", {}).get("BLOCOS_EXPLORATORIOS_SIRGASPolygon.shp")
    gdf_campos_producao = process_zip_source_shp("campos_producao", data_sources).get("geodataframes", {}).get("CAMPOS_PRODUCAO_SIRGASPolygon.shp")

    print("\nDatasets prontos para análise:")
    datasets = {
        "df_sismica_2023_orig": df_sismica_2023_orig,
        "df_pocos_orig": df_pocos_orig,
        "df_lev_geoq_2022_orig": df_lev_geoq_2022_orig,
        "df_geoq_2021_orig": df_geoq_2021_orig,
        "df_reservas_orig": df_reservas_orig,
        "df_poco_2023_orig": df_poco_2023_orig,
        "gdf_setores_sirgas": gdf_setores_sirgas,
        "gdf_blocos_exploratorios": gdf_blocos_exploratorios,
        "gdf_campos_producao": gdf_campos_producao,
    }
    for name, dataset in datasets.items():
        if dataset is not None:
            print(f"- {name}: Shape {dataset.shape}")
    return datasets

In [94]:
def main():
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Inicializar proveniência
    doc_prov, dict_agents, dict_activities, dict_entities = initProvenance()

    # Carregar datasets e GeoDataFrames
    datasets = create_eda_dataset()

    # Salvar os DataFrames e GeoDataFrames no diretório de saída
    saved_objects = save_dataframes(globals())

    # Listar objetos salvos
    if saved_objects:
        print("\nObjetos salvos:")
        for obj_name in saved_objects:
            print(f"- {obj_name}")
    else:
        print("\nNenhum objeto foi salvo.")

    # Serializar e gerar saídas de proveniência
    gerar_prov_outputs(doc_prov)

In [95]:
main()

Dados carregados com sucesso para 'levantamento_sismico_2023'.
Dados carregados com sucesso para 'tabela_pocos_2024'.
Dados carregados com sucesso para 'tabela_levantamentos_geoquimica'.
Dados carregados com sucesso para 'tabela_dados_geoquimica'.
Dados carregados com sucesso para 'reservas_nacionais_hc'.
Dados carregados com sucesso para 'pocos_perfurados_2023'.
Conteúdo do ZIP analisado: {'csv': [], 'xlsx': [], 'xls': [], 'shp': [{'name': 'SETORES_TODOS_SIRGAS.shp', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.shp'}], 'others': [{'name': 'SETORES_TODOS_SIRGAS.dbf', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.dbf'}, {'name': 'SETORES_TODOS_SIRGAS.prj', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.prj'}, {'name': 'SETORES_TODOS_SIRGAS.sbn', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.sbn'}, {'name': 'SETORES_TODOS_SIRGAS.sbx', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.sbx'}, {'name': 'SETORES_TODOS_SIRGAS.shx', 'path': './dados/temp\\SETORES_TODOS_SIRGAS.shx'}, {'name': 'SETORES_TODOS_

In [776]:
#df_sismica_2023_orig = load_data_from_source("levantamento_sismico_2023", data_sources)
#df_sismica_2023_orig.columns

In [12]:
"""def inspect_dataframes(dataframes):
    
    #Inspeciona um dicionário de DataFrames, exibindo colunas, shape e estatísticas descritivas.
   
    for name, df in dataframes.items():
        print(f"\n=== {name} ===")
        print(f"Shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print("\nHead:")
        print(df.head())
        print("\nDescribe:")
        print(df.describe(include='all', datetime_is_numeric=True))

    def clean_dataframe_columns(dataframes):
  
    #Realiza o saneamento e a renomeação de colunas em um conjunto de DataFrames.

    cleaned_dataframes = {}
    for name, df in dataframes.items():
        # Renomear colunas removendo espaços e convertendo para letras minúsculas
        df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

        # Registrar atividade de saneamento na proveniência
        execStartTime = datetime.datetime.now()
        activity_key = f"act-saneamento-{name}"
        dict_activities[activity_key] = doc_prov.activity(f"ufrj:saneamento_{name}", execStartTime, None, {
            "prov:label": escape_label(f"Saneamento de colunas: {name}")
        })
        doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

        entity_key = f"ent-saneado-{name}"
        dict_entities[entity_key] = doc_prov.entity(f"ufrj:{name}_saneado", {
            "prov:label": escape_label(f"Dataset saneado: {name}"),
            "prov:type": "void:Dataset",
        })
        doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])

        cleaned_dataframes[name] = df
    return cleaned_dataframes
        """